In [9]:
from gpkit import Model, Variable, units
import gpkit
from numpy import pi
gpkit.settings["latex_modelname"] = False

class ltaHALE(Model):
    def setup(self):

        constraints = []

        # Constants
        g = Variable('g', 9.81,'m/s^2', 'gravitational constant')
        R_earth = Variable('R_{earth}', 6371*10^3, 'meters', 'radius of the earth')
        R = Variable('R', 287, 'J/kg*K', 'ideal gas constant')

        # Requirements
        m_pay = Variable('m_{pay}', 10, 'lb', 'mass of payload') 
        Thresh_Diameter = Variable('Thresh_{diam}', 100, 'km', 'threshold footprint diameter')
        Object_Diameter = Variable('Object_{diam}', 300, 'km', 'objective footprint diameter')
        Inclination_Angle = Variable('angle_{incl}', 5, 'deg', 'max antenna inclination angle')
        time = Variable('t', 5*24, 'hr', 'flight time')
        P_payload = Variable('P_{pay}', 10, 'W', 'power requirement of payload')

        # Assumptions
        Cd = Variable('C_d', 0.04, '-', 'drag coefficient')
        eta_prop = Variable(r'\eta_{prop}', 0.75, '-', 'propulsive efficiency')
        m_bat = Variable('m_{bat}', 2, 'kg', 'mass of payload battery')
        m_avi = Variable('m_{avi}', 1, 'kg', 'mass of avionics')
        m_fixed = Variable('m_{fixed}', 'lbm', 'fixed mass')

        constraints.extend([m_fixed >= m_pay + m_avi + m_bat])

        f_fixed_pay = Variable('f_{weightMassFraction}', 0.1, '-', 'Fixed weight mass fraction')

        W = Variable('W', 'lbf', 'vehicle weight')
        W_diesel = Variable('W_{diesel}', 'lbf', 'mission diesel fuel weight')
        m_balloon = Variable('m_ballo0n', 'kg', 'mass of balloon skin (surface area * skin thickness * density)')
        constraints.extend([W >= m_fixed * g/f_fixed_pay + W_diesel + m_balloon * g])

#         # # Atmospheric Stuff -- MATLAB syntax
#         T=zeros(1,30000); # Celsius Temperature Matrix 
#         P=zeros(1,30000); # Kilopascal Pressure Matrix
#         for h=1:30000
#             if h<11000
#                 T(1,h)=15.04-0.00649*h;
#                 P(1,h)=101.29*((T(1,h)+273.1)/288.08)^5.256;
#             elseif h<25000 && h>=11000
#                 T(1,h)=-56.46;
#                 P(1,h)=22.65*exp(1.73-0.000157*h);
#             elseif h>=25000
#                 T(1,h)=-131.21+0.00299*h;
#                 P(1,h)=2.488*((T(1,h)+273.1)/216.6)^-11.388;
#             end
#         end

#         p=P./(0.2869.*(T+273.1)); # # kg/m^3 Density Matrix
#         h=1:30000;

#         # # Altitude of Flight accounting for Curvature of the Earth (Assumed Spherical Earth)
#         Theta = Variable(r'\Theta', Object_Diameter / (4 * pi * R_earth) * 360, 'deg', 'theta')
#         Chord_earth = Variable('Chord_{earth}', 2 * R_earth * sind(Theta/2), 'm', 'chord length of the earth')
#         Alpha = Variable(r'\Alpha', (180 - Theta)/2, 'deg', 'alpha')
#         Beta = Variable(r'\Beta', 180 - Alpha, 'deg', 'beta')
#         Zeta = Variable(r'\Zeta', 90 - Alpha, 'deg', 'zeta')
#         Gamma = Variable(r'\Gamma' 180 - Beta - Inclination_Angle - Zeta, 'deg', 'gamma')

#         constraints.extend([H >= Chord_earth * sind(Inclination_Angle + Zeta)/sind(Gamma)]); # # meters

#         # # Environment
        rho = Variable(r'\rho', 0.4, 'kg/m^3', 'air density')
#         press = P(1,H); # pressure for this selected altitude
#         temp = T(1,H); # # temperature for this selected altitude
        vel = Variable('vel', 'm/s', 'cruise velocity')
        v_wind = Variable('v_{wind}', 35, 'm/s', 'wind velocity')

        constraints.extend([vel >= v_wind])

#         # # LTA Sizing
        vol = Variable('vol', 'm^3', 'vehicle Volume')
        t = Variable('thickness ratio', 0.5, '-', 'thickness ratio')
#         c = Variable('chord', (Vol/(10 * pi * t * (2/3 * 0.2969 - 0.126 * 0.5 - 0.3516 * 1/3 + 0.2843 * 1/4 - 0.1015 * 1/5)))^(1/3), 'm', 'chord length')
        area = Variable('area', 'm^2', 'frontal area')
        r = Variable('r', 'ft', 'vehicle radius')
        rho_he = Variable(r'\rho_{he}', 0.1786, 'g/L', 'density of helium at stp')
        rho_h = Variable(r'\rho_{h}', 0.08988, 'g/L', 'density of hydrogen at stp')
        t_skin = Variable('t_{skin}', 0.125, 'in', 'thickness of skin of balloon')
        rho_skin = Variable(r'\rho_{skin}', 1.45, 'g/cm^3', 'density of polyester skin')
        constraints.extend([vol >= W/(rho * g),
                            vol == 4/3. * pi * r**3,
                            area == 4 * pi * r**2,
                            m_balloon == t_skin* area * rho_skin])

        # Power Consumption
#         CD = Variable('C_D', '-', 'Drag coefficient')
#         CL = Variable('C_L', '-', 'Lift coefficient')
        P_shaft = Variable('P_{shaft}', 'W', 'Shaft power')
#         S = Variable('S', 'm^2', 'Wing reference area')
#         V = Variable('V', 'm/s', 'Cruise velocity')
#         V_min = Variable('V_{min}', 15, "m/s", "minimum flight velocity")
#         W = Variable('W', 'lbf', 'Aircraft weight')
        
        constraints.extend([P_shaft >= 1 /eta_prop/2. * rho * vel**3 * Cd * area])

#         # # Energy Needed
#         E_engines = Variable('E_{engines}', P_shaft * time, 'W * hr', 'engine energy')
#         E_payload = Variable('E_{payload}', P_payload * time, 'W * hr', 'payload energy')

        E_total = Variable('E_{total}', "MJ", "total mission energy")

#         # Carbon Fuels
        h_diesel = Variable('h_{diesel}', 35.86, 'MJ/L', 'diesel energy density')
        Diesel_eff = Variable(r'\Eta_{diesel}', 0.5, '-', 'diesel engine efficiency')
        Diesel_dens = Variable(r'\Rho_{diesel}', 0.832, 'kg/L', 'diesel density')
        constraints.extend([W_diesel >= E_total/h_diesel*Diesel_dens*g,
                            E_total >= (P_payload + P_shaft)*time])
#         Gasoline_Edens = Variable('gasoline_{edens}', 32.4, 'MJ/L', 'gasoline energy density')
#         Gasoline_eff = Variable(r'\Eta_{gasoline}', 0.25, '-', 'gasoline engine efficiency')
#         Gasoline_dens = Variable(r'\Rho_{gasoline}', 0.75, 'kg/L', 'gasoline density')
#         Jetfuel_Edens = Variable('jetfuel_{edens}', 37.4, 'MJ/L', 'jetfuel energy density')
#         Jetfuel_dens = Variable(r'\Rho_{jetfuel}', 0.81, 'kg/L', 'jetfuel density')
#         constraints.extend([Diesel_needed >= E_total_joules/(Diesel_Edens * 10^6),
#                             Gasoline_needed >= E_total_joules/(Gasoline_Edens * 10^6),
#                             Jetfuel_needed >= E_total_joules/(Jetfuel_Edens * 10^6),
#                             Diesel_mass >= Diesel_needed*Diesel_dens,
#                             Gasoline_mass >= Gasoline_needed * Gasoline_dens,
#                             Jetfuel_mass >= Jetfuel_needed * Jetfuel_dens,
#                             Diesel_fraction >= Diesel_mass/(W/g),
#                             Gasoline_fraction >= Gasoline_mass/(W/g),
#                             Jetfuel_fraction >= Jetfuel_mass/(W/g)])

        # Atmosphere model
        h = Variable("h", 10000, "ft", "Altitude")
        p_sl = Variable("p_{sl}", 101325, "Pa", "Pressure at sea level")
        T_sl = Variable("T_{sl}", 288.15, "K", "Temperature at sea level")
        L_atm = Variable("L_{atm}", 0.0065, "K/m", "Temperature lapse rate")
        T_atm = Variable("T_{atm}", "K", "air temperature")
        M_atm = Variable("M_{atm}", 0.0289644, "kg/mol", "Molar mass of dry air")
        R_atm = Variable("R_{atm}", 8.31447, "J/mol/K")
        TH = (g*M_atm/R_atm/L_atm).value.magnitude  # dimensionless
        constraints.extend([h <= 20000*units.m,  # Model valid to top of troposphere
                            T_sl >= T_atm + L_atm*h,     # Temp decreases w/ altitude
                            rho <= p_sl*T_atm**(TH-1)*M_atm/R_atm/(T_sl**TH)])  # http://en.wikipedia.org/wiki/Density_of_air#Altitude
        
#         # station keeping requirement
#         footprint = Variable("d_{footprint}", 200, 'km', "station keeping footprint diameter")
#         lu = Variable(r"\theta_{look-up}", 5, '-', "look up angle")
#         R_earth = Variable("R_{earth}", 6371, "km", "Radius of earth")
#         tan_lu = lu*pi/180. + (lu*pi/180.)**3/3.  # Taylor series expansion
#         # approximate earth curvature penalty as distance^2/(2*Re)
#         constraints.extend([h >= tan_lu*0.5*footprint + footprint**2/8./R_earth])
        
        objective = W

        return objective, constraints

m = ltaHALE()
m.solve('cvxopt')


Using solver 'cvxopt'
Solving for 11 variables.
Solving took 0.0229 seconds.

Cost
----
 1.209e+06 [lbf] 

Free Variables
--------------
 E_{total} : 1.179e+07  [MJ]   total mission energy                                          
 P_{shaft} : 2.73e+07   [W]    Shaft power                                                   
   T_{atm} : 264.2      [K]    air temperature                                               
         W : 1.209e+06  [lbf]  vehicle weight                                                
W_{diesel} : 6.033e+05  [lbf]  mission diesel fuel weight                                    
      area : 5.968e+04  [m**2] frontal area                                                  
 m_ballo0n : 2.748e+05  [kg]   mass of balloon skin (surface area * skin thickness * density)
 m_{fixed} : 16.61      [lb]   fixed mass                                                    
         r : 226.1      [ft]   vehicle radius                                                
       vel : 35  

{'constants': {p_{sl}_ltaHALE7: array(101325),
  h_ltaHALE7: array(10000),
  T_{sl}_ltaHALE7: array(288.15),
  L_{atm}_ltaHALE7: array(0.0065),
  M_{atm}_ltaHALE7: array(0.0289644),
  R_{atm}_ltaHALE7: array(8.31447),
  h_{diesel}_ltaHALE7: array(35.86),
  \Rho_{diesel}_ltaHALE7: array(0.832),
  g_ltaHALE7: array(9.81),
  m_{pay}_ltaHALE7: array(10),
  \rho_ltaHALE7: array(0.4),
  t_{skin}_ltaHALE7: array(0.125),
  v_{wind}_ltaHALE7: array(35),
  m_{avi}_ltaHALE7: array(1),
  \eta_{prop}_ltaHALE7: array(0.75),
  t_ltaHALE7: array(120),
  P_{pay}_ltaHALE7: array(10),
  m_{bat}_ltaHALE7: array(2),
  C_d_ltaHALE7: array(0.04),
  f_{weightMassFraction}_ltaHALE7: array(0.1),
  \rho_{skin}_ltaHALE7: array(1.45)},
 'cost': array(1209481.877562737),
 'freevariables': {area_ltaHALE7: array(59683.641503114944),
  r_ltaHALE7: array(226.10374819948456),
  P_{shaft}_ltaHALE7: array(27295318.71410106),
  T_{atm}_ltaHALE7: array(264.24445760323977),
  E_{total}_ltaHALE7: array(11791582.004493421),
  